In [1]:
from datasets import load_dataset

ds = load_dataset("openai/gsm8k", "main")

print(ds)

/Users/arpan/Desktop/ExplAIn/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1319/1319 [00:00<00:00, 364758.16 examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})


In [8]:
ds['train']

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

In [6]:
# train_grpo.py
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer

dataset = load_dataset("trl-lib/tldr", split="train")

# # Define the reward function, which rewards completions that are close to 20 characters
# def reward_len(completions, **kwargs):
#     return [-abs(20 - len(completion)) for completion in completions]

# training_args = GRPOConfig(output_dir="Qwen2-0.5B-GRPO", logging_steps=10)
# trainer = GRPOTrainer(
#     model="Qwen/Qwen2-0.5B-Instruct",
#     reward_funcs=reward_len,
#     args=training_args,
#     train_dataset=dataset,
# )
# trainer.train()

Generating test split: 100%|██████████| 6553/6553 [00:00<00:00, 270316.82 examples/s]


In [7]:
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 116722
})

In [13]:
gsm8k_train = ds['train']
# Rename the first column to prompt and format it as an instruction
gsm8k_train = gsm8k_train.map(lambda x: {
    'prompt': f"Please solve this math problem step by step. Show your reasoning and put the final answer in <answer></answer> tags.\n\nProblem: {x['question']}"
})



Map: 100%|██████████| 7473/7473 [00:00<00:00, 47457.81 examples/s]


In [14]:
gsm8k_train[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
 'prompt': 'Please solve this math problem step by step. Show your reasoning and put the final answer in <answer></answer> tags.\n\nProblem: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'}

In [9]:
# Define the reward function, which rewards completions that are close to 20 characters
def reward_len(completions, **kwargs):
    print(f"completions: {completions}")
    print(f"kwargs: {kwargs}")
    return [-abs(20 - len(completion)) for completion in completions]

training_args = GRPOConfig(output_dir="Qwen2-0.5B-GRPO", logging_steps=10)
trainer = GRPOTrainer(
    model="Qwen/Qwen2-0.5B-Instruct",
    reward_funcs=reward_len,
    args=training_args,
    train_dataset=ds['train'],
)
trainer.train()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
/Users/arpan/Desktop/ExplAIn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyError: 'prompt'